[View in Colaboratory](https://colab.research.google.com/github/rdenadai/jupyter-pool/blob/master/nomes.ipynb)

## Nomes

Notebook que irá coletar todos os nomes de pessoas existentes no site [Dicionario de Nomes Próprios](https://www.dicionariodenomesproprios.com.br/).

O programa abaixo recebe como parâmetro o tamanho do nome que se quer apresentar, por padrão, só retorna nomes com quantidade de letras igual a 7. Por exemplo: Eduardo, Alberto.

In [92]:
!apt-get install libxml2-dev libxslt-dev
!pip install aiomultiprocess aiohttp cchardet aiodns numba lxml

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libxslt1-dev' instead of 'libxslt-dev'
libxslt1-dev is already the newest version (1.1.29-2.1ubuntu1).
libxml2-dev is already the newest version (2.9.4+dfsg1-4ubuntu1.2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [0]:
import asyncio
import functools
import concurrent.futures
from collections import namedtuple
from bs4 import BeautifulSoup as bs
import aiohttp
from aiomultiprocess import Pool
from numba import jit
import numpy as np
import pandas as pd

In [0]:
# Nome = namedtuple('Nome', ['href', 'nome'])

async def fetch(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            # print(url)
            return await resp.text()


async def start(urls):
    responses = []
    async with Pool() as pool:
        responses = await pool.map(fetch, urls)
    return responses


def salvar_nomes(filename, nomes):
    with open(filename, 'w') as h:
        for nome in nomes:
            h.write(f'{nome}\n')


def parse_names(response, name_size=7):
    data = []
    parser = bs(response, 'lxml')
    links = parser.find_all('a', class_='lista-nome')
    for link in links:
        # data.append(Nome(href=link['href'], nome=link.string))
        nome = link.string.strip()
        if len(nome) == name_size:
            data.append(nome)
    return '|'.join(data)


@jit(parallel=True)
def name_split(stringNomes):
    print('Realizando split dos nomes...')
    nomes = np.array([])
    for lre in stringNomes:
        nomes = np.concatenate((nomes, lre.split('|')), axis=None)
    yield filter(None, nomes.tolist())
    

def retorna_nomes(sexo='M', name_size=7, pages=150):
    nomes = []
    try:
        if sexo.upper() == 'M':
            urls = [f'https://www.dicionariodenomesproprios.com.br/nomes-masculinos/{i}' for i in range(1, pages)]
        elif sexo.upper() == 'F':
            urls = [f'https://www.dicionariodenomesproprios.com.br/nomes-femininos/{i}' for i in range(1, pages)]
        loop = asyncio.get_event_loop()
        responses = loop.run_until_complete(asyncio.gather(start(urls)))[0]
        print('Terminado o download das páginas...')
        with concurrent.futures.ProcessPoolExecutor(max_workers=4) as exc:
            print('Iniciando parser das páginas...')
            result = exc.map(functools.partial(parse_names, name_size=name_size), responses, chunksize=5)
            print('Extraindo nomes do resultado...')
            nomes = name_split(result)
    except KeyboardInterrupt:
        loop.stop()
    return sorted(list(nomes)[0])


def side_by_side(M, N):
    return [M[i:i+N] for i in range(0, len(M), N)]

In [95]:
pages = 150
print('-' * 20)
print('Executando rotina (sexo masculino)...')
M = retorna_nomes(sexo='M', pages=pages)
print('\n')
print('-' * 20)
print('Executando rotina (sexo feminino)...')
F = retorna_nomes(sexo='F', pages=pages)

--------------------
Executando rotina (sexo masculino)...
Terminado o download das páginas...
Iniciando parser das páginas...
Extraindo nomes do resultado...
Realizando split dos nomes...


--------------------
Executando rotina (sexo feminino)...
Terminado o download das páginas...
Iniciando parser das páginas...
Extraindo nomes do resultado...
Realizando split dos nomes...


In [101]:
N = 20
print('Apresentando resultados...')
print('-' * 20)
print('Nomes Masculinos:')
df = pd.DataFrame(side_by_side(M, N), columns=['' for _ in range(N)])
df[df.isnull()] = ''
display(df)
print('\n')
print('Nomes Femininos:')
df = pd.DataFrame(side_by_side(F, N), columns=['' for _ in range(N)])
df[df.isnull()] = ''
display(df)
# print('Salvar arquivos...')
# print('Salvando nomes masculinos...')
# salvar_nomes('nomes_masculinos.txt', M)
# print('Salvando nomes femininos...')
# salvar_nomes('nomes_femininos.txt', F)

Apresentando resultados...
--------------------
Nomes Masculinos:


,,,,,,,,,,,,,,,,,,,,
0,Absalom,Absirto,Accioli,Acelino,Achille,Acrísio,Acúrcio,Adalard,Adaliah,Adalton,Adelino,Adelmar,Adelson,Aderbal,Aderico,Adonias,Adriano,Adílson,Afrânio,Agledes
1,Agnaldo,Aguilar,Aguimar,Aguirre,Agustin,Aimberê,Ajagunã,Aladino,Alarico,Alberto,Alceste,Alcindo,Alcídes,Alencar,Alfonso,Alfredo,Allyson,Almeida,Aloísio,Aluísio
2,Amintas,Amorreu,Amâncio,Amândio,Américo,Amílcar,Ananias,Anatole,Andrade,Aniceto,Anselmo,Antelmo,Antenor,Anthony,Antoine,Antonin,Antunes,Antônio,Apollon,Aprígio
3,Aquiles,Arantes,Araquém,Araripe,Araruna,Arcádio,Arduíno,Ariosto,Aristeu,Arlindo,Armando,Armindo,Armênio,Armínio,Arnaldo,Arnoldo,Arnóbio,Arouche,Arsênio,Artêmio
4,Ascânio,Asenate,Assuero,Astride,Ataulfo,Augusto,Aurélio,Avelino,Azarias,Azevedo,Bacelar,Balbino,Baldwin,Barnabé,Barreto,Bartolo,Baruque,Basileu,Basílio,Batimeu
5,Batista,Belmiro,Belquis,Beltrão,Ben-Hur,Benigno,Benício,Beraldo,Bertino,Bertoli,Bertram,Berílio,Birigui,Bismark,Bolívar,Bourbon,Brandon,Brandão,Brendon,Briaréu
6,Brizola,Bronwen,Brígido,Burnier,Caetano,Calisto,Calixto,Calvino,Camarão,Canindé,Cardoso,Carioca,Carlito,Carlton,Carmelo,Caronte,Castiel,Caxambu,Cecílio,Cercion
7,Chantal,Charles,Charlie,Chester,Cirineu,Ciríaco,Clayton,Cleiton,Cleyton,Cleófas,Clinton,Cláudio,Coimbra,Colombo,Conrado,Corinto,Crispim,Crispin,Curvelo,Cândido
8,Dalmiro,Damiano,Damásio,Delfino,Delmiro,Deodato,Deodoro,Dercino,Diaurum,Dickson,Dimitre,Dimitri,Dinarte,Diocles,Dominic,Domício,Donaldo,Doriano,Dorival,Douglas
9,Edgardo,Edilson,Edmundo,Ednaldo,Edoardo,Eduardo,Edvaldo,Egberto,Eistein,Eleakim,Eleazar,Eliézer,Elpídio,Emanuel,Emerson,Enrique,Ephraim,Epêneto,Ercílio,Ermínio




Nomes Femininos:


,,,,,,,,,,,,,,,,,,,,
0,Aaliyah,Aaminah,Abayomi,Abigail,Acemira,Acilina,Acirema,Adalina,Adamina,Adelina,Adeline,Adelita,Adozina,Adriana,Adriane,Adriele,Adônica,Agenora,Akeneki,Alarice
1,Alawara,Alberta,Alcinda,Alcione,Alcmena,Alcídia,Aldaíra,Alegria,Alekena,Aleluia,Alenica,Alepeka,Alessia,Alethea,Alethia,Alisson,Allegra,Allende,Alméria,Althaia
2,Amalina,Amanacy,Amanacy,Amanara,Amapola,Ambrose,Amábile,América,Anabela,Analisa,Andresa,Andreza,Andrija,Andréia,Anelise,Anemone,Angícia,Anninka,Anselma,Anselme
3,Antelma,Antônia,Anúncia,Apolina,Arabela,Araceli,Aracema,Aretusa,Ariadne,Arianna,Arianne,Aricema,Aricina,Arienne,Aritana,Arlinda,Arminda,Arriaga,Arrieta,Arriola
4,Arsione,Artêmia,Ashiley,Aspásia,Assunta,Astéria,Ataléia,Augusta,Aurélia,Avelina,Azucena,Açucena,Baepeba,Balbina,Bambina,Baraúna,Barbela,Barbosa,Bartira,Batilde
5,Bedelia,Belfort,Belinda,Belmira,Bendita,Benigna,Benilda,Benoite,Benícia,Berdine,Bernice,Bersabé,Bertina,Bethany,Betsabé,Bettina,Betânia,Beverly,Bibiana,Blanche
6,Bridget,Brigita,Brigite,Brunnah,Brígida,Bárbara,Cacilda,Cacéres,Caillet,Caitlin,Caitria,Calista,Calíope,Camapuã,Cambuci,Cameron,Camille,Camilly,Camocim,Campbel
7,Camélia,Canirim,Caprina,Carlota,Carmela,Cassidy,Catrina,Cecília,Celeste,Cendira,Ceressa,Chandra,Chelsea,Clarice,Clarina,Clarita,Cleanto,Cláudia,Coaraci,Colares
8,Colomba,Cordula,Corália,Cristal,Cynthia,Cândida,Dagmara,Dakarai,Dalmira,Dalvina,Damiana,Dandara,Daniela,Daniele,Danúbia,Darlene,Deborah,Dejanir,Delamar,Delfina
9,Demeter,Destiny,Deífila,Diacira,Diadema,Diomara,Dionira,Djamila,Djamira,Dolores,Dominga,Dorothy,Drielle,Dâmaris,Désirée,Edelina,Edeline,Edilene,Edinéia,Ednalva
